In [1]:
import json
import pandas as pd
import re
import random
import numpy as np
from nltk.corpus import stopwords 
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

######## METRICS

from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import precision_recall_curve

from sklearn.metrics import classification_report

from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve


# matplotlib

import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
REPLACE_NO_SPACE = re.compile("[.;:!\'?,\"()\[\]]")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
def preprocess_reviews(reviews):
    reviews = [REPLACE_NO_SPACE.sub("", line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(" ", line) for line in reviews]

    return reviews

In [3]:
def classify_it(filename, parametr): 
    i = 0
    reviews_lst_good= []
    reviews_lst_bad = []

    #набираем список хороших/плохих отзывов, не более 500000 в каждом из списков
    
    with open(filename, 'r') as f:
            for line in f:
                if json.loads(line)['overall'] > 3 and len(reviews_lst_good)<=75000:
                    reviews_lst_good.append (json.loads(line))

                elif json.loads(line)['overall'] < 3 and len(reviews_lst_bad)<=75000:
                    reviews_lst_bad.append (json.loads(line))

                if len(reviews_lst_good) + len (reviews_lst_bad) == 150000:
                    break
    # random elements in the list
    
    # перемешиваем список
    
    random.shuffle(reviews_lst_good)
    random.shuffle(reviews_lst_bad)  
    
    # создаём template для сохранения конечных отзывов
    reviewerID_lst = []
    reviewerName_lst = []
    reviewText_lst = []
    overall_lst = []
    summary_lst = []
    
    # где будем уменьшать количество отзывов, дабы их было равное количество
    
    if len (reviews_lst_bad) < len(reviews_lst_good):
        count_otz = len (reviews_lst_bad)
    else:
        count_otz = len (reviews_lst_good)

    # обрезаем хороший
    for i in reviews_lst_good[:count_otz]:
        reviewText_lst.append (i['reviewText'])
        overall_lst.append (i['overall'])

    print ('Good: ',len(reviews_lst_good[:count_otz]))

    # обрезаем плохие
    for i in reviews_lst_bad[:count_otz]:
        reviewText_lst.append (i['reviewText'])
        overall_lst.append (i['overall'])
        
    print ('Bad: ',len(reviews_lst_bad[:count_otz]))
    
#     test_len = int(round (len(reviews_lst_bad[:count_otz])*0.1,0))
#     #длина выборки, на которой строим тесты  
#     print ('10%: ',test_len)

    df = pd.DataFrame ()

    
    df['reviewText'] = reviewText_lst
    df['overall'] = overall_lst
    
    # 0 и 1 ставим
    df ['mark'] = 0
    df.loc[df['overall'] > 3,'mark'] = 1
    
    # количество отзывов в нормальной и тестовой выборке
    reviews_train = list(df[df['overall'] > 3]['reviewText']) + list(df[df['overall'] < 3]['reviewText'])
    y = list(df[df['overall'] > 3]['mark']) + list(df[df['overall'] < 3]['mark'])
    
    # очищаем отзывы от всяких лишних символов
    reviews_train_clean = preprocess_reviews(reviews_train)
    
    
    
    
      # делим выборку на тестовую и обучающую
    X_train, X_val, y_train, y_val = train_test_split(
        reviews_train_clean, y, train_size = 0.75
    )  
    
    
    


    
    # столбец ответов 
    
    
    

    
    
    print ('Подбор параметра для: ', filename[:-5])
    print ('    ')
    
    
    best_c_dict = {}
    main_dict = {}
    c_list = []
    result_1_list = []
    result_2_list = []
    
    # определяем, где алгоритму необходимо искать С
    grid = [1]
    
        
    # бинаризируем столбец признаков
    cv = TfidfVectorizer(min_df=10, decode_error='replace', encoding='utf-8')
    X = cv.fit_transform(X_train)
    
    X_test = cv.transform(X_val)
    
    # итерируемся и подбираем параметр, используя Accuracy
    for c in grid:
        final_model = svm.SVC(kernel='linear', C=c)
        final_model.fit(X, y_train)
        
        y_pred = final_model.predict(X_test)
        
        print ("Accuracy for C=%s: %s" 
               % (c, accuracy_score(y_val, y_pred)))
        
        c_list.append (c)
        result_1_list.append (accuracy_score(y_val, y_pred))

        best_c_dict [c] = accuracy_score(y_val, y_pred)
        main_dict [c] = [filename, accuracy_score(y_val, y_pred)]
        
    
    # записываем результаты
    result = pd.DataFrame()
    result['c'] = c_list
    result ['result_1'] = result_1_list
    result.to_csv (filename[:-5]+'_podbor_c_svm.csv', index=False)
    ###############
    
    inverse = [(value, key) for key, value in best_c_dict.items()]
    print ('The best C is ', max(inverse)[1])

    # находим лучший результат и записываем его + используем для модели
    
#     final_model = svm.SVC(kernel='linear',C=max(inverse)[1])
    print ('So model use C = ', max(inverse)[1])
#     final_model.fit(X,  y_train)
    print ("Final Accuracy: %s" 
           % accuracy_score(y_val
                            , y_pred))
    
    print('confusion_matrix')
    from sklearn.metrics import confusion_matrix
    confusion_matrix = confusion_matrix(y_val, y_pred)
    print(confusion_matrix)
    
    print ('classification_report')
    print(classification_report(y_val, y_pred))
    
    
#     print ('roc crivai')
    

#     logit_roc_auc = roc_auc_score(y_val, final_model.predict(X_test))
#     fpr, tpr, thresholds = roc_curve(y_val, final_model.predict_proba(X_test)[:,1])
#     plt.figure()
    
    
#     data = {'fpr': fpr, 'tpr': tpr}
#     df = pd.DataFrame.from_dict(data)
#     df.to_csv (filename[:-5]+'_roc_crivai_svm.csv', index=False)
    
#     plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
#     plt.plot([0, 1], [0, 1],'r--')
#     plt.xlim([0.0, 1.0])
#     plt.ylim([0.0, 1.05])
#     plt.xlabel('False Positive Rate')
#     plt.ylabel('True Positive Rate')
#     plt.title('Receiver operating characteristic')
#     plt.legend(loc="lower right")
#     plt.savefig('Log_ROC')
#     plt.show()
    
    # выделяем слова, которые обладают самым большим весом
    feature_to_coef = {
    word: coef for word, coef in zip(
        cv.get_feature_names(), final_model.coef_[0]
        )
    }
    for best_positive in sorted(
        feature_to_coef.items(), 
        key=lambda x: x[1], 
        reverse=True)[:25]:
        print (best_positive)
    

#     word_lst = []
#     score_lst = []
#     for i in best_positive:
#         word_lst.append (i)
#         score_lst.append (best_positive[i])
    
#     data = {'word': word_ls, 'level': score_lst}
#     df = pd.DataFrame.from_dict(data)
#     df.to_csv (filename[:-5]+'_posit_words.csv', index=False)
    
    for best_negative in sorted(
        feature_to_coef.items(), 
        key=lambda x: x[1])[:25]:
        print (best_negative)
    
#     word_lst = []
#     score_lst = []
#     for i in best_negative:
#         word_lst.append (i)
#         score_lst.append (best_negative[i])
    
#     data = {'word': word_ls, 'level': score_lst}
#     df = pd.DataFrame.from_dict(data)
#     df.to_csv (filename[:-5]+'_negat_words.csv', index=False)
        
    del reviews_lst_good
    del reviews_lst_bad
    
    return main_dict

In [ ]:
import os
import gc

os.listdir('/Volumes/GoogleDrive/Мой диск/Work/Experiment')

for i in os.listdir('/Volumes/GoogleDrive/Мой диск/Work/Experiment'):
    if i[-5:] != 'ipynb'and i[-3:] != 'csv' and i != '.ipynb_checkpoints': # and i == 'Health_and_Personal_Care_5.json':
        print (i[:-5])
        print ('   ')
        dict_main = classify_it ('/Volumes/GoogleDrive/Мой диск/Work/Experiment/' + i, 0.25)
        gc.collect()
        

Health_and_Personal_Care_5
   
Good:  33300
Bad:  33300


/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Подбор параметра для:  /Volumes/GoogleDrive/Мой диск/Work/Experiment/Health_and_Personal_Care_5
    
Accuracy for C=1: 0.8674474474474474
The best C is  1
So model use C =  1
Final Accuracy: 0.8674474474474474
confusion_matrix
[[7323 1059]
 [1148 7120]]
classification_report
              precision    recall  f1-score   support

           0       0.86      0.87      0.87      8382
           1       0.87      0.86      0.87      8268

   micro avg       0.87      0.87      0.87     16650
   macro avg       0.87      0.87      0.87     16650
weighted avg       0.87      0.87      0.87     16650

('00', <1x11300 sparse matrix of type '<class 'numpy.float64'>'
	with 11295 stored elements in Compressed Sparse Row format>)
('00', <1x11300 sparse matrix of type '<class 'numpy.float64'>'
	with 11295 stored elements in Compressed Sparse Row format>)
Movies_and_TV_5
   
Good:  74999
Bad:  74999
Подбор параметра для:  /Volumes/GoogleDrive/Мой диск/Work/Experiment/Movies_and_TV_5
    


In [ ]:
# import numpy as np
# import matplotlib.mlab as mlab
# import matplotlib.pyplot as plt
# %matplotlib inline

# x_lst = []
# y_lst = []
# for i in dict_main:
#     x_lst.append(i)
#     y_lst.append(dict_main[i][1])

# plt.bar(x_lst, y_lst, align='center', alpha=0.3, width=0.1)

In [ ]:
result = pd.DataFrame( = [1,2,3], b = [4,5,6])
result

Comments

https://habr.com/ru/company/ods/blog/323890/#1-lineynaya-regressiya

https://docplayer.ru/28834116-Otchet-po-zadaniyu-4-issledovanie-modeli-logistic-regression.html

https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression

http://www.machinelearning.ru/wiki/index.php?title=%D0%9C%D0%B0%D1%88%D0%B8%D0%BD%D0%BD%D0%BE%D0%B5_%D0%BE%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D0%B5_(%D0%BA%D1%83%D1%80%D1%81_%D0%BB%D0%B5%D0%BA%D1%86%D0%B8%D0%B9,_%D0%9A.%D0%92.%D0%92%D0%BE%D1%80%D0%BE%D0%BD%D1%86%D0%BE%D0%B2)#.D0.9E.D1.81.D0.BD.D0.BE.D0.B2.D0.BD.D1.8B.D0.B5_.D0.BF.D0.BE.D0.BD.D1.8F.D1.82.D0.B8.D1.8F_.D0.B8_.D0.BF.D1.80.D0.B8.D0.BC.D0.B5.D1.80.D1.8B_.D0.BF.D1.80.D0.B8.D0.BA.D0.BB.D0.B0.D0.B4.D0.BD.D1.8B.D1.85_.D0.B7.D0.B0.D0.B4.D0.B0.D1.87



ROC:
https://basegroup.ru/community/articles/logistic

https://machinelearningmastery.com/roc-curves-and-precision-recall-curves-for-classification-in-python/

https://towardsdatascience.com/building-a-logistic-regression-in-python-step-by-step-becd4d56c9c8